<a target="_blank" href="https://colab.research.google.com/github/segmed/gemma_mistral/blob/main/notebooks/datasets_example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
!nvidia-smi

Sat Feb 24 17:26:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# if you are using google colab

import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
if os.environ["HF_TOKEN"] == None:
    from huggingface_hub import notebook_login
    notebook_login()

In [3]:
!pip install -qq "transformers==4.38.0" --upgrade
!pip -qq install bitsandbytes accelerate datasets pandas peft trl tqdm evaluate
!pip -qq install "torch>=2.1.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━

In [4]:
from datasets import load_dataset
dataset = load_dataset('medmcqa', split='train').shuffle(seed=42).train_test_split(test_size=0.01)
dataset

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 180993
    })
    test: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 1829
    })
})

In [5]:
import pandas as pd
df_train = pd.DataFrame(dataset['train'])
df_train.head(1)

,id,question,opa,opb,opc,opd,cop,choice_type,exp,subject_name,topic_name
0,0114fd9e-52e5-40ca-a0b4-fb3c146d275f,A herd immunity of over .........is considered...,50%,55%,60%,70%,3,single,70%,Social & Preventive Medicine,None


In [6]:
import pandas as pd
df_test = pd.DataFrame(dataset['test'])
df_test.head(1)

,id,question,opa,opb,opc,opd,cop,choice_type,exp,subject_name,topic_name
0,acd5220c-7b55-4bd4-bdd4-4cffe23eefb7,Which mineral causes skin allergy: March 2013,Magnesium,Zinc,Copper,Nickel,3,multi,Ans. D i.e. Nickel,Skin,None


In [7]:
df_train['subject_name'].value_counts()

Medicine                        17718
Surgery                         16660
Pathology                       14740
Anatomy                         14417
Pharmacology                    13609
Social & Preventive Medicine    11772
Microbiology                    11200
Gynaecology & Obstetrics         9903
Dental                           8852
Physiology                       8741
Biochemistry                     8194
Pediatrics                       7959
Ophthalmology                    6881
Forensic Medicine                5837
ENT                              4869
Psychiatry                       4403
Radiology                        4355
Anaesthesia                      3134
Unknown                          3025
Orthopaedics                     2969
Skin                             1755
Name: subject_name, dtype: int64

In [8]:
df_train['question'].apply(len).mean()

78.73387921079821

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model_name = "google/gemma-7b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             quantization_config=quantization_config)

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [10]:
import numpy as np

choice_map = np.array(["(a)", "(b)", "(c)", "(d)"])

def get_content(row):
    question = row['question']
    cop = row['cop']
    opa = row['opa']
    opb = row['opb']
    opc = row['opc']
    opd = row['opd']
    exp = row['exp']
    topic_name = row['topic_name']
    optiona = f"(a) {opa}"
    optionb = f"(b) {opb}"
    optionc = f"(c) {opc}"
    optiond = f"(d) {opd}"
    exp = row['exp']
    answer = choice_map[cop]

    system_prompt_choice = """Analyze the question and determine which choice is the correct one and return the answer as one or more of the following "(a)", "(b)", "(c)", "(d)" followed by an explanation:"""
    return f"{system_prompt_choice}\n{question}\n{optiona} {optionb} {optionc} {optiond}", answer, exp

def get_test_sample(row):
    content, answer, exp = get_content(row)
    chat = [
        { "role": "user", "content": content },
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True, add_special_tokens=False)
    return prompt

def get_test_sample_answer(row):
    content, answer, exp = get_content(row)
    return answer, exp

def get_training_sample(row):
    content, answer, exp = get_content(row)
    chat = [
        { "role": "user", "content": content },
        { "role": "model", "content": f"{answer}\n{exp}" },
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False, add_special_tokens=False)
    return prompt

In [19]:
for index, row in df_train.loc[:1].iterrows():
    sample = get_training_sample(row)
    display(sample)

'<bos><start_of_turn>user\nAnalyze the question and determine which choice is the correct one and return the answer as one or more of the following "(a)", "(b)", "(c)", "(d)" followed by an explanation:\nA herd immunity of over .........is considered necessary to prevent epidemic spread of diphtheria:\n(a) 50% (b) 55% (c) 60% (d) 70%<end_of_turn>\n<start_of_turn>model\n(d)\n70%<end_of_turn>\n'

'<bos><start_of_turn>user\nAnalyze the question and determine which choice is the correct one and return the answer as one or more of the following "(a)", "(b)", "(c)", "(d)" followed by an explanation:\nThe mother of a one and a half year old child gives history of a white reflex from one eye for the past 1 month. On computed tomography scan of the orbit, there is calcification seen within the globe. The most likely diagnosis is –\n(a) Congenital cataract (b) Retinoblastoma (c) Endophthalmitis (d) Coats disease<end_of_turn>\n<start_of_turn>model\n(b)\nWhile pupillary reflex (leukocoria) with calcification in tumor mass is characteristic of retinoblastoma.<end_of_turn>\n'

In [20]:
for index, row in df_test.loc[:1].iterrows():
    sample = get_test_sample(row)
    display(sample)

'<bos><start_of_turn>user\nAnalyze the question and determine which choice is the correct one and return the answer as one or more of the following "(a)", "(b)", "(c)", "(d)" followed by an explanation:\nWhich mineral causes skin allergy: March 2013\n(a) Magnesium (b) Zinc (c) Copper (d) Nickel<end_of_turn>\n<start_of_turn>model\n'

'<bos><start_of_turn>user\nAnalyze the question and determine which choice is the correct one and return the answer as one or more of the following "(a)", "(b)", "(c)", "(d)" followed by an explanation:\nA completed denture patient complains that he has pain in his throat during swallowing. This indicates:\n(a) Over extended PPS (b) Over extended lingual flange (c) Over extended Lingual flange in retromylohyoid region (d) His tonsils are inflammed<end_of_turn>\n<start_of_turn>model\n'

In [21]:
df_train['text'] = df_train.apply(get_training_sample, axis=1)
df_test['text'] = df_test.apply(get_test_sample, axis=1)

In [34]:
def predict(row, model, tokenizer):
    y_pred = None
    prompt = get_test_sample(row)
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=64, do_sample=True, top_k=0, num_return_sequences=1, temperature=0.01, eos_token_id=tokenizer.eos_token_id)
    result = tokenizer.decode(outputs[0])
    answer = result.split("<start_of_turn>model")[-1].lower()
    y_pred = "none"
    for index, choice in enumerate(choice_map):
        if choice in answer:
            y_pred = choice

    return y_pred, input_ids['input_ids'].shape[1]

In [36]:
for index, row in df_test.loc[:3].iterrows():
    choice, explanation = get_test_sample_answer(row)
    prediction, num_tokens = predict(row, model, tokenizer)
    display(f"prediction - {prediction}, reference {choice}")

'prediction - (d), reference (d)'

'prediction - (c), reference (c)'

'prediction - (b), reference (b)'

'prediction - (d), reference (c)'

In [47]:
from tqdm import tqdm
import time

references = []
predictions = []
times = []
num_tokens = []

for index, row in tqdm(df_test.loc[:100].iterrows(), total=df_test.shape[0]):
    seconds = time.time()
    choice, explanation = get_test_sample_answer(row)
    reference_index = np.where(choice_map == choice)[0][0]
    references.append(reference_index)
    prediction, num_token = predict(row, model, tokenizer)
    prediction_index = np.where(choice_map == prediction)[0][0]
    predictions.append(prediction_index)
    times.append(time.time() - seconds)
    num_tokens.append(num_token)

  6%|▌         | 101/1829 [11:46<3:21:21,  6.99s/it]


In [48]:
import evaluate

accuracy_metric = evaluate.load("accuracy")
results = accuracy_metric.compute(predictions=predictions, references=references)
display("accuracy:", results["accuracy"])

'accuracy:'

0.44554455445544555

In [49]:
average = sum(times) / len(times)
display("duration per sample:", round(average, 2))

'duration per sample:'

6.99

In [51]:
average = sum(num_tokens) / len(num_tokens)
display("avg tokens per sample:", round(average, 2))

'avg tokens per sample:'

94.09

In [52]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             quantization_config=quantization_config)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [53]:
from tqdm import tqdm
import time

references = []
predictions = []
times = []
num_tokens = []

for index, row in tqdm(df_test.loc[:100].iterrows(), total=df_test.shape[0]):
    seconds = time.time()
    choice, explanation = get_test_sample_answer(row)
    reference_index = np.where(choice_map == choice)[0][0]
    references.append(reference_index)
    prediction, num_token = predict(row, model, tokenizer)
    prediction_index = np.where(choice_map == prediction)[0][0]
    predictions.append(prediction_index)
    times.append(time.time() - seconds)
    num_tokens.append(num_token)

  6%|▌         | 101/1829 [10:56<3:07:08,  6.50s/it]


In [54]:
import evaluate

accuracy_metric = evaluate.load("accuracy")
results = accuracy_metric.compute(predictions=predictions, references=references)
display("accuracy:", results["accuracy"])

'accuracy:'

0.2079207920792079

In [55]:
average = sum(times) / len(times)
display("duration per sample:", round(average, 2))

'duration per sample:'

6.5

In [56]:
average = sum(num_tokens) / len(num_tokens)
display("avg tokens per sample:", round(average, 2))

'avg tokens per sample:'

104.23

In [57]:
from google.colab import runtime
runtime.unassign()